In [29]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os
import imageio
import tqdm
import time

In [30]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


<h2> Create an A matrix with T x N (time x mean firing rate)

<h3> Get Policy mjrl

In [31]:
policy = "./iterations/best_policy.pickle" #Change the folder to the policy location

import pickle
# load policy
pi = pickle.load(open(policy, 'rb'))


<h2> Setup data collection for mjrl policy </h2>

In [32]:
# Use sleep if you want to examine certain steps during the episodes
from time import sleep
import torch

env = gym.make('CenterReachOut-v0')
frames = [] 

TotalNeuralData = []
TotalMuscleActData = []
TotalFiberJointData = []
TotalFiberData = []
TotalJointData = []

total_reward = 0

# This is the number of samples per condition
num_samples = 10

ep_num = 0


for x in range(num_samples):
  NeuralData = {}
  MuscleActData = {}
  FiberJointData = {}
  FiberData = {}
  JointData = {}
  conditions = {}

  for n in range(120):  #Number of episodes
    obs = env.reset()

    condition = env.obs_dict['obj_pos'][1]

    if condition in conditions:
      continue
  
    if len(conditions) == 8:
      break
    
    conditions[condition] = 1
    
    
    timestep = 0
    done = False
    
    NeuralData[condition] = []
    MuscleActData[condition] = []
    FiberJointData[condition] = []
    FiberData[condition] = []
    JointData[condition] = []


    for ac_num in range(16): # Number of steps per episode / Each time step is 0.05 seconds
        timestep += 0.05
        frame = env.sim.renderer.render_offscreen(
                          width=400,
                          height=400,
                          camera_id=1)
        frames.append(frame)
        # We're appending data of each timestep to an epsiode number key
        o = env.get_obs()
        action, _ = pi.get_action(o)

        # Adding some noise to the action
        #action = add_noise(action)
        
        # For the mjrl policy 
        # For MLPs
        # NeuralData[condition].append(np.concatenate((pi.show_activations()["layer_0"][0],pi.show_activations()["layer_1"][0] )))

        # For LSTM (Both layers)
        NeuralData[condition].append(np.concatenate((pi.show_activations()[0].detach().numpy()[0][0],pi.show_activations()[1].detach().numpy()[0])))
        
        # For LSTM (Only the recurrent layer)
        #NeuralData[condition].append(pi.show_activations()[0].detach().numpy()[0][0])

        #NeuralData[condition].append(torch.cat((model.policy.activations[0], model.policy.activations[1]), dim=0).flatten().cpu().numpy())
        MuscleActData[condition].append(env.obs_dict['act'])
        FiberJointData[condition].append(np.concatenate((env.obs_dict['hand_qpos'],env.obs_dict['hand_qvel'], env.sim.data.actuator_length[:].copy(), env.sim.data.actuator_velocity[:].copy())))
        FiberData[condition].append(np.concatenate((env.obs_dict['hand_qpos'], env.obs_dict['hand_qvel'])))
        JointData[condition].append(np.concatenate((env.sim.data.actuator_length[:].copy(), env.sim.data.actuator_velocity[:].copy())))

        obs, reward, done, info = env.step(action)
        total_reward += reward
        
    ep_num += 1
  
  TotalNeuralData.append(NeuralData)
  TotalMuscleActData.append(MuscleActData)
  TotalFiberJointData.append(FiberJointData)
  TotalJointData.append(JointData)
  TotalFiberData.append(FiberData)

            
env.close()

print(total_reward)

os.makedirs('videos', exist_ok=True)
video_path = 'videos/test_policy.mp4'
# make a local copy
imageio.mimsave(video_path, frames, fps=30)
show_video('videos/test_policy.mp4')

-7676.714736156117


# Average Code

In [33]:
def average_data(TotalData):
    averaged_data = {}
    
    # Iterate over all episodes
    for episode_data in TotalData:
        for condition, values in episode_data.items():
            # Convert to NumPy arrays for numerical operations
            values = np.array(values)

            if condition not in averaged_data:
                averaged_data[condition] = []

            averaged_data[condition].append(values)

    # Compute the mean for each condition
    for condition in averaged_data:
        averaged_data[condition] = np.mean(averaged_data[condition], axis=0)

    # Re-map keys from conditions to sequential integers (0, 1, 2, ...)
    new_keys = {old_key: new_index for new_index, old_key in enumerate(sorted(averaged_data.keys()))}
    averaged_data = {new_keys[old_key]: values for old_key, values in averaged_data.items()}

    return averaged_data


# Compute averages
NeuralData = average_data(TotalNeuralData)
MuscleActData = average_data(TotalMuscleActData)
FiberJointData = average_data(TotalFiberJointData)
FiberData = average_data(TotalFiberData)
JointData = average_data(TotalJointData)

In [34]:
print(f"Neural Data: {NeuralData} \n length: {len(NeuralData[1][4])}") # 512 Neurons
print(f"Muscle Actuator Data: {MuscleActData[1]}\n length: {len(MuscleActData[1])}") # 63 Muscles
print(f"Fiber/Joint Angle/Joint Velocity Data: {FiberJointData[1]} \n length: {len(FiberJointData[1])}") # 63 muscles + 44 joints

Neural Data: {4: array([[-0.3728, -0.2671,  0.2927, ...,  0.8291,  0.4799, -0.0911],
       [-0.3958, -0.1648,  0.3344, ...,  0.6537, -0.1014, -0.6804],
       [-0.3511, -0.1934,  0.4427, ...,  0.7633, -0.0368, -0.5534],
       ...,
       [-0.3067, -0.2264,  0.4065, ...,  0.5858, -0.2171, -0.4544],
       [-0.3055, -0.2409,  0.4055, ...,  0.5695, -0.245 , -0.4552],
       [-0.301 , -0.2385,  0.4003, ...,  0.5514, -0.2607, -0.4618]],
      dtype=float32), 6: array([[-0.3728, -0.2624,  0.3008, ...,  0.8068,  0.3307, -0.1915],
       [-0.3952, -0.1904,  0.3494, ...,  0.7015, -0.0126, -0.607 ],
       [-0.358 , -0.211 ,  0.4393, ...,  0.807 ,  0.1194, -0.379 ],
       ...,
       [-0.2956, -0.2311,  0.4022, ...,  0.5522, -0.2529, -0.4229],
       [-0.2914, -0.2314,  0.3963, ...,  0.5259, -0.2548, -0.4219],
       [-0.2883, -0.2326,  0.3914, ...,  0.5177, -0.2611, -0.3978]],
      dtype=float32), 1: array([[-3.6607e-01, -2.5235e-01,  3.0309e-01, ...,  8.0490e-01,
         3.2933e-01, -2.46

In [35]:
import scipy.io

time_steps = 16

# Create the `times` array and ensure it's double precision
times =  np.arange(0, time_steps*0.05, 0.05).reshape(-1, 1)
    
data = [NeuralData, MuscleActData, FiberJointData, FiberData, JointData]
matrices = []

for x in range(len(data)): # 3 Matrices
    data_struct = np.empty((7,), dtype=[('A', 'O'), ('times', 'O')])

    for n in range(7):
        data_struct['A'][n] = data[x][n] 
        data_struct['times'][n] = times  

    
    matrices.append(data_struct)
    
# Save the structured data to a .mat file with 'Data' as the matrix name
scipy.io.savemat('./jPCAData/MLP_w_noise_neural_data.mat', {'Data': matrices[0]})
scipy.io.savemat('./jPCAData/MLP_w_noise_muscle_data.mat', {'Data': matrices[1]})
scipy.io.savemat('./jPCAData/MLP_w_noise_data.mat', {'Data': matrices[2]})


<p> We'll basically have 3 matrices A (number of rows = number of steps): <br>
    - Neurons: 512 columns for each neuron <br>
    - Muscle Activity: 63 columns for each muscle actuator intensity <br>
    - Fiber Length/Joint Angle/Velocity: 151 columns for combination of 63 fiber length, 44 joint angle and 44 joint velocities <br>

# Down Sampling
<p> Since the amount of neurons, muscles and fiber/joints are not equal, we want to make that we account for that size difference. 

In [36]:
print(len(data[2][0][0]))
# neural data: len(data[0][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 512
# muscle data: len(data[1][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 63
# fiber data: len(data[2][ep_1, ep_2 ... ep108][act_1, act_2...act16]) = 214

test_neuraldata = list(data[0].values())
test_muscledata = list(data[1].values())
test_fiberdata = list(data[2].values())

214


In [37]:
def downsample(dataset):
    downsampled_dataset = []
    for episode in dataset:
        downsampled_episode = []
        for activity in episode:
            if len(activity) > 63:
                step = len(activity) // 63
                downsampled_activity = activity[::step][:63]  
            else:
                downsampled_activity = activity 
            downsampled_episode.append(downsampled_activity)
        downsampled_dataset.append(downsampled_episode)
    return downsampled_dataset

In [38]:
downsampled_Neural_data = {}

downsampled_Neural_data = downsample(test_neuraldata)

downsampled_Neural_data = {index: value for index, value in enumerate(downsampled_Neural_data)}

In [39]:
downsampled_FiberJoint_data = {}

downsampled_FiberJoint_data = downsample(test_fiberdata)
    
downsampled_FiberJoint_data = {index: value for index, value in enumerate(downsampled_FiberJoint_data )}

print(len(downsampled_Neural_data[0][0]))

63


Create Matrix

In [40]:
import numpy as np
import scipy.io

time_steps = 16

# Create the `times` array and ensure it's double precision
times =  np.arange(0, time_steps*0.05, 0.05).reshape(-1, 1)
    
data = [downsampled_Neural_data, MuscleActData, downsampled_FiberJoint_data]
matrices = []

for x in range(3): # 3 Matrices
    data_struct = np.empty((ep_num,), dtype=[('A', 'O'), ('times', 'O')])

    for n in range(ep_num):
        data_struct['A'][n] = data[x][n] 
        data_struct['times'][n] = times  

    
    matrices.append(data_struct)
    
# Save the structured data to a .mat file with 'Data' as the matrix name
scipy.io.savemat('Neural_Data_downsampled_training.mat', {'Data': matrices[0]})
scipy.io.savemat('MuscleAct_Data_downsampled_training.mat', {'Data': matrices[1]})
scipy.io.savemat('FiberJoint_Data_downsampled_training.mat', {'Data': matrices[2]})
# scipy.io.savemat('Fiber_Data_downsampled_training.mat', {'Data': matrices[3]})
# scipy.io.savemat('Joint_Data_downsampled_training.mat', {'Data': matrices[4]})


KeyError: 8